## Requesting multiple series and specific dates from BLS API

In [ ]:
import pandas as pd
import requests
import json
import prettytable
import config 
from config import BLS_API_KEY

# The url for BLS API v2
url = 'https://api.bls.gov/publicAPI/v1/timeseries/data/'

# API key in config.py which contains: bls_key = 'key'
key = '?registrationkey={}'.format(config.BLS_API_KEY)

# Series stored as a dictionary
series_dict = {
    'LEU0252917300': 'High sch grads no coll, 25 yrs & ovr',
    'LEU0254929400': 'Some coll or assc degr, 25 yrs & ovr ',
    'LEU0252919100': 'Bach degr only, 25 yrs & ovr ',
    'LEU0252918500': 'Bach degr or higher, 25 yrs & ovr ',
    'LEU0252919700': 'Adv degree, 25 yrs & ovr '}

# Start year and end year
dates = ('2014', '2019')

# Specify json as content type to return
headers = {'Content-type': 'application/json'}

# Submit the list of series as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})

# Post request for the data
p = requests.post('{}{}'.format(url, key),
    headers=headers,
    data=data).json()['Results']['series']


# Loop through series and convert to pandas format

In [ ]:
# use for loop to apply formatting or other changes to all series in the returned json data
# Date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

# Create a dataframe to fill with values
df = pd.DataFrame()

# Build a pandas series from the API results, p
for s in p:
    df[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]
    # Show last 5 results
df.tail()

# Plot the results

In [ ]:
# Simple line plot of the data
# df.plot(title='Earnings by Educational Attainment')

# Conclusion

In [ ]:
# what does the line plot show?

In [ ]:
import pandas as pd
import json
import requests

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['LEU0252918500'],"startyear":"2014", "endyear":"2019"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
try:
    df = pd.DataFrame()
    for series in json_data['Results']['series']:
        df_initial = pd.DataFrame(series)
        series_col = df_initial['seriesID'][0]
        for i in range(0, len(df_initial) - 1):
            df_row = pd.DataFrame(df_initial['data'][i])
            df_row['seriesID'] = series_col
            if 'code' not in str(df_row['footnotes']): 
                df_row['footnotes'] = ''
            else:
                df_row['footnotes'] = str(df_row['footnotes']).split("'code': '",1)[1][:1]
            df = df.append(df_row, ignore_index=True)
    df.to_csv('blsdata.csv', index=False)
except:
    json_data['status'] == 'REQUEST_NOT_PROCESSED'
    print('BLS API has given the following Response:', json_data['status'])
    print('Reason:', json_data['message'])